# CS246 - Colab 9
## Studying COVID-19

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

# 新段落

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be analyzing the time series data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [6]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

In [7]:
confirmed.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

### Your Task

We are aware of the stress we are all experiencing because of the still-ongoing pandemic and the fact that many of you have projects and exams due this week. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [8]:
# YOUR CODE HERE
c = confirmed.select(sum("5/1/21")).collect()[0][0]
d = deaths.select(sum("5/1/21")).collect()[0][0]
r = recovered.select(sum("5/1/21")).collect()[0][0]
c, d, r

(152196159.0, 3192930.0, 88919401.0)

Consider the data points for March 1, 2020, and March 1, 2021, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of deaths and number of confirmed cases.

In [9]:
# YOUR CODE HERE
#confirmed.show()
selected = confirmed.select('Province/State', 'Country/Region', 'Lat', 'Long', '3/1/20','3/1/21')
selected = selected.where(col('3/1/21') >= 50)

dt = deaths.select('Lat', 'Long', '3/1/20','3/1/21')
dt = dt.withColumnRenamed("3/1/20", "A")\
       .withColumnRenamed("3/1/21", "B")
selected = selected.join(dt, (dt.Long == selected.Long) & (dt.Lat == selected.Lat), 'inner')
selected.show()


+--------------------+-------------------+---------+----------+------+-------+---------+----------+---+-----+
|      Province/State|     Country/Region|      Lat|      Long|3/1/20| 3/1/21|      Lat|      Long|  A|    B|
+--------------------+-------------------+---------+----------+------+-------+---------+----------+---+-----+
|                null|        Afghanistan| 33.93911| 67.709953|     1|  55733| 33.93911| 67.709953|  0| 2444|
|                null|            Albania|  41.1533|   20.1683|     0| 107931|  41.1533|   20.1683|  0| 1816|
|                null|            Algeria|  28.0339|    1.6596|     1| 113255|  28.0339|    1.6596|  0| 2987|
|                null|            Andorra|  42.5063|    1.5218|     0|  10889|  42.5063|    1.5218|  0|  110|
|                null|             Angola| -11.2027|   17.8739|     0|  20854| -11.2027|   17.8739|  0|  508|
|                null|Antigua and Barbuda|  17.0608|  -61.7964|     0|    769|  17.0608|  -61.7964|  0|   14|
|         

In [10]:
selected = selected.withColumn("Ratio",selected[5]/selected[9])
selected.show()

+--------------------+-------------------+---------+----------+------+-------+---------+----------+---+-----+------------------+
|      Province/State|     Country/Region|      Lat|      Long|3/1/20| 3/1/21|      Lat|      Long|  A|    B|             Ratio|
+--------------------+-------------------+---------+----------+------+-------+---------+----------+---+-----+------------------+
|                null|        Afghanistan| 33.93911| 67.709953|     1|  55733| 33.93911| 67.709953|  0| 2444|22.804009819967266|
|                null|            Albania|  41.1533|   20.1683|     0| 107931|  41.1533|   20.1683|  0| 1816|59.433370044052865|
|                null|            Algeria|  28.0339|    1.6596|     1| 113255|  28.0339|    1.6596|  0| 2987| 37.91596919986608|
|                null|            Andorra|  42.5063|    1.5218|     0|  10889|  42.5063|    1.5218|  0|  110| 98.99090909090908|
|                null|             Angola| -11.2027|   17.8739|     0|  20854| -11.2027|   17.873

Consider the data points for March 1, 2021, and May 1, 2021, in the timeseries, and filter out the geographical locations where less than 50 deaths have been confirmed (as of March 1, 2021).
For the areas still taken into consideration after the filtering step, compute the percent increase in cumulative deaths between the two dates.

In [11]:
# YOUR CODE HERE
selected = confirmed.select('Province/State', 'Country/Region', 'Lat', 'Long', '3/1/21','5/1/21')
selected = selected.where(col('3/1/21') >= 50)

dt = deaths.select('Lat', 'Long', '3/1/21','5/1/21')
dt = dt.withColumnRenamed("3/1/21", "A")\
       .withColumnRenamed("5/1/21", "B")
selected = selected.join(dt, (dt.Long == selected.Long) & (dt.Lat == selected.Lat), 'inner')
selected.show()


+--------------------+-------------------+---------+----------+-------+-------+---------+----------+-----+-----+
|      Province/State|     Country/Region|      Lat|      Long| 3/1/21| 5/1/21|      Lat|      Long|    A|    B|
+--------------------+-------------------+---------+----------+-------+-------+---------+----------+-----+-----+
|                null|        Afghanistan| 33.93911| 67.709953|  55733|  59939| 33.93911| 67.709953| 2444| 2631|
|                null|            Albania|  41.1533|   20.1683| 107931| 131185|  41.1533|   20.1683| 1816| 2396|
|                null|            Algeria|  28.0339|    1.6596| 113255| 122311|  28.0339|    1.6596| 2987| 3261|
|                null|            Andorra|  42.5063|    1.5218|  10889|  13232|  42.5063|    1.5218|  110|  125|
|                null|             Angola| -11.2027|   17.8739|  20854|  26815| -11.2027|   17.8739|  508|  600|
|                null|Antigua and Barbuda|  17.0608|  -61.7964|    769|   1232|  17.0608|  -61.7

In [12]:
selected = selected.withColumn("Increase",(selected.B - selected.A)/selected.A)
selected.select('A','B','Increase').show()

+-----+-----+-------------------+
|    A|    B|           Increase|
+-----+-----+-------------------+
| 2444| 2631| 0.0765139116202946|
| 1816| 2396|0.31938325991189426|
| 2987| 3261|0.09173083361232005|
|  110|  125|0.13636363636363635|
|  508|  600|0.18110236220472442|
|   14|   32| 1.2857142857142858|
|52077|64096|0.23079286441231253|
| 3195| 4128|  0.292018779342723|
|    3|    3|                0.0|
|   54|   54|                0.0|
|    0|    0|               null|
|    6|    7|0.16666666666666666|
|    4|    4|                0.0|
|   13|   13|                0.0|
|  820|  820|                0.0|
|    9|    9|                0.0|
| 8574|10233|0.19349195241427572|
| 3223| 4538| 0.4080049643189575|
|  179|  199|0.11173184357541899|
|  452|  648| 0.4336283185840708|
+-----+-----+-------------------+
only showing top 20 rows



Once you have working code for each cell above, **head over to Gradescope, read the questions carefully, and submit your solution for this Colab**!